# Import selenium và webdriver

* selenium - webdriver
* Beautiful soup
* time
* webdriver-manager để auto xử lý chrome driver
 

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from webdriver_manager.chrome import ChromeDriverManager 

import time
import os
import json
import random
import re
import pickle

from bs4 import BeautifulSoup

# Đăng nhập và cấu hình driver

In [5]:
SCREEN_WIDTH = 1920
SCREEN_HEIGHT = 1080
BATCH_SIZE = 25

def setup_driver():
    chrome_options = Options()
    # Thêm đường dẫn tới Chrome browser
    chrome_options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"  # Đường dẫn mặc định của Chrome
    
    chrome_options.add_argument("--app")
    chrome_options.add_argument(f"--window-size={SCREEN_WIDTH},{SCREEN_HEIGHT}")
    chrome_options.add_argument("--window-position=0,0")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36")
    prefs = {"profile.managed_default_content_settings.images": 2}
    chrome_options.add_experimental_option("prefs", prefs)
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    return driver

# Đăng nhập vào web

* account: dihidromonooxit01012000@gmail.com
* password: 1!Aaaaaaaaaa

In [6]:
def sign_in(driver, email, password):
    """
    Đăng nhập vào ITviec và lưu cookies để duy trì session
    Returns:
        bool: True nếu đăng nhập thành công, False nếu thất bại
    """
    cookies_path = os.path.join('cookies', 'itviec_cookies.pkl')
    
    # Thử load cookies trước
    if os.path.exists(cookies_path):
        driver.get("https://itviec.com")
        load_cookies(driver, cookies_path)
        driver.refresh()  # Refresh để áp dụng cookies
        
        if is_logged_in(driver):
            print("Đã đăng nhập thành công bằng cookies!")
            return True
        else:
            print("Cookie không có tác dụng gì")
    
    # Nếu không có cookies hoặc cookies hết hạn, đăng nhập bình thường
    print("Tiến hành đăng nhập...")
    driver.get("https://itviec.com/sign_in")
    time.sleep(2)

    try:
        email_input = driver.find_element(By.NAME, "user[email]")
        password_input = driver.find_element(By.NAME, "user[password]")

        email_input.send_keys(email)
        password_input.send_keys(password)

        submit_button = driver.find_element(By.XPATH, "//button[@class='ibtn ibtn-md ibtn-primary w-100']")
        submit_button.click()

        # Đợi cho đến khi đăng nhập thành công (tối đa 10 giây)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "sign-in-user-avatar"))
        )
        
        # Lưu cookies sau khi đăng nhập thành công
        save_cookies(driver, cookies_path)
        print("Đăng nhập thành công!")
        return True

    except Exception as e:
        print(f"Lỗi khi đăng nhập: {str(e)}")
        return False
    
        
    email_input = driver.find_element(By.NAME, "user[email]")
    password_input = driver.find_element(By.NAME, "user[password]")

    email_input.send_keys(email)
    password_input.send_keys(password)

    submit_button = driver.find_element(By.XPATH, "//button[@class='ibtn ibtn-md ibtn-primary w-100']")
    submit_button.click()
    
    time.sleep(3)

    driver.get("https://itviec.com/it-jobs")
    # time.sleep(2)
    # Đợi cho đến khi đăng nhập thành công (tối đa 10 giây)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "sign-in-user-avatar"))
    )
    try:
        # Lưu cookies sau khi đăng nhập thành công
        save_cookies(driver, cookies_path)
        print("Đã save cookie")
        return True

    except Exception as e:
        print(f"Lỗi khi đăng nhập: {str(e)}")
        return False

In [7]:
def save_cookies(driver, path):
    """Lưu cookies của session hiện tại vào file"""
    if not os.path.exists('cookies'):
        os.makedirs('cookies')
    with open(path, 'wb') as f:
        pickle.dump(driver.get_cookies(), f)
    print(f"Đã lưu cookies vào {path}")

def load_cookies(driver, path):
    """Load cookies từ file vào driver"""
    if not os.path.exists(path):
        print(f"Không tìm thấy file cookies: {path}")
        return False
    
    with open(path, 'rb') as f:
        cookies = pickle.load(f)
        for cookie in cookies:
            # Giả sử 'cookie' là một dictionary chứa thông tin cookie
            # if 'domain' in cookie:
            #     del cookie['domain']
            # if 'expiry' in cookie: # Khóa 'expiry' cũng thường gây lỗi
            #     del cookie['expiry']
            driver.add_cookie(cookie)
    print(f"Đã load cookies từ {path}")
    return True

def is_logged_in(driver):
    """Kiểm tra xem đã đăng nhập chưa"""
    try:
        # Đợi tối đa 3 giây để tìm element chỉ xuất hiện khi đã đăng nhập
        # Ví dụ: link "Sign Out" hoặc tên user
        driver.get("https://itviec.com")
        WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.CLASS_NAME, "sign-in-user-avatar"))
        )
        return True
    except TimeoutException:
        return False

# Test cào thử cái gì đó


In [ ]:
def test_crawl(driver):
   
    driver.get("https://itviec.com/it-jobs")
    # driver.get("https://itviec.com")
    cookies_path = os.path.join('cookies', 'itviec_cookies.pkl')
    load_cookies(driver, cookies_path)
    
    time.sleep(5)
    test = driver.page_source
    print("Truy cập trang chủ ITviec thành công!")
    return test

In [13]:
driver =setup_driver()
sign_in(driver, "dihidromonooxit01012000@gmail.com", "1!Aaaaaaaaaa")
content = test_crawl(driver)
# current_cookie = driver.get_cookies()
# print("cookie:", current_cookie)
soup = BeautifulSoup(content, 'html.parser')
# pattern_x = re.compile(r'\b' + re.escape('salary') + r'\b')
job_cards = soup.find('div', class_='d-flex align-items-center salary text-success-color')
print(job_cards)
# avatar = soup.find('div', class_='sign-in-user-avatar')
# avatar


Đã load cookies từ cookies\itviec_cookies.pkl
Đã đăng nhập thành công bằng cookies!
Đã load cookies từ cookies\itviec_cookies.pkl
Truy cập trang chủ ITviec thành công!
<div class="d-flex align-items-center salary text-success-color">
<svg fill="none" height="20" stroke="currentColor" viewbox="0 0 24 24" width="20" xmlns="http://www.w3.org/2000/svg">
<path d="M12.0044 6V18" stroke-linecap="round" stroke-linejoin="round" stroke-width="2"></path>
<path d="M14.9287 7.90332H10.5395C9.99625 7.90332 9.47527 8.11912 9.09114 8.50325C8.70701 8.88738 8.49121 9.40836 8.49121 9.9516C8.49121 10.4948 8.70701 11.0158 9.09114 11.4C9.47527 11.7841 9.99625 11.9999 10.5395 11.9999H13.4656C14.0088 11.9999 14.5298 12.2157 14.914 12.5998C15.2981 12.9839 15.5139 13.5049 15.5139 14.0482C15.5139 14.5914 15.2981 15.1124 14.914 15.4965C14.5298 15.8806 14.0088 16.0964 13.4656 16.0964H8.49121" stroke="currentColor" stroke-linecap="round" stroke-linejoin="round" stroke-width="2"></path>
<circle cx="12.0002" cy="12.0

In [10]:
content

'<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml"><head><style type="text/css" data-tag-name="trix-editor">trix-editor {\n    display: block;\n}\n\ntrix-editor:empty::before {\n    content: attr(placeholder);\n    color: graytext;\n    cursor: text;\n    pointer-events: none;\n    white-space: pre-line;\n}\n\ntrix-editor a[contenteditable=false] {\n    cursor: text;\n}\n\ntrix-editor img {\n    max-width: 100%;\n    height: auto;\n}\n\ntrix-editor [data-trix-attachment] figcaption textarea {\n    resize: none;\n}\n\ntrix-editor [data-trix-attachment] figcaption textarea.trix-autoresize-clone {\n    position: absolute;\n    left: -9999px;\n    max-height: 0px;\n}\n\ntrix-editor [data-trix-attachment] figcaption[data-trix-placeholder]:empty::before {\n    content: attr(data-trix-placeholder);\n    color: graytext;\n}\n\ntrix-editor [data-trix-cursor-target] {\n    display: inline-block !important;\n    width: 1px !important;\n    padding: 0 !important;\n    margin: 0 !i

# Tìm kiếm tất cả các link title công việc

In [15]:
# def page_next(driver):
#     next_button = driver.find_element(By.XPATH, "//a[@rel='next']")
#     next_button.click()
#     time.sleep(random.uniform(2, 4))
#     print("Chuyển sang trang tiếp theo thành công!")
def page_next(driver):
    try:
        # 1. Chờ đợi tường minh (Explicit Wait) cho đến khi nút 'next' có thể click được
        # Đảm bảo element xuất hiện và không bị che
        next_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@rel='next']"))
        )
        
        # 2. Sử dụng JavaScript để click
        # Cách này thường đáng tin cậy hơn driver.click()
        driver.execute_script("arguments[0].click();", next_button)
        
        # 3. Chờ đợi sau khi click
        time.sleep(random.uniform(3, 5)) # Tăng thời gian chờ để đảm bảo trang tải xong
        
        print("Chuyển sang trang tiếp theo thành công!")
        return True # Trả về True nếu thành công

    except Exception as e:
        # Nếu không tìm thấy nút trong 10 giây (hoặc lỗi khác)
        print("Không tìm thấy nút 'Trang tiếp theo' hoặc hết trang.")
        # print("Lỗi chi tiết:", e) # Có thể bỏ dòng này nếu không cần gỡ lỗi
        return False # Trả về False nếu thất bại

hàm lấy tất cả các link job 

In [16]:
def crawl_links(driver):
    driver.get("https://itviec.com/it-jobs")
    time.sleep(random.uniform(2, 3))
    links = []
    while True:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        job_cards = soup.find_all('div', class_='job-card')
        for job_card in job_cards:
            # Tìm kiếm tất cả các link công việc trong chính thẻ đó
            job_url_attribute = 'data-search--job-selection-job-url-value'
            
            if job_url_attribute in job_card.attrs:
                relative_link = job_card.attrs[job_url_attribute]
                relative_link = relative_link.split('/content')[0] # loại bỏ phần dư thừa để truy cập vào đúng link 
                full_link = "https://itviec.com" + relative_link
                links.append(full_link)
            else:
                print("ERROR: Không tìm thấy thuộc tính link trong thẻ job-card.")
                
        print("------------------------------------")
        print('đang ở trang số:', len(links)//BATCH_SIZE)
        print("Đã thu thập được", len(links), "liên kết công việc.")
        try:
            next_page = soup.find('a', rel='next')
            if next_page and next_page['href']:
                driver.get("https://itviec.com" + next_page['href'])
                time.sleep(random.uniform(2, 3))
            else:
                print("Không tìm thấy trang tiếp theo. Kết thúc thu thập liên kết.")
                break
            
        except Exception as e:
            print("Đã hết trang hoặc có lỗi xảy ra:", e)
            break
        
    return links

In [17]:
test = crawl_links(driver)
print(test)
print(len(test))

------------------------------------
đang ở trang số: 0
Đã thu thập được 20 liên kết công việc.
------------------------------------
đang ở trang số: 1
Đã thu thập được 40 liên kết công việc.
------------------------------------
đang ở trang số: 1
Đã thu thập được 40 liên kết công việc.
------------------------------------
đang ở trang số: 2
Đã thu thập được 60 liên kết công việc.
------------------------------------
đang ở trang số: 2
Đã thu thập được 60 liên kết công việc.
------------------------------------
đang ở trang số: 3
Đã thu thập được 80 liên kết công việc.
------------------------------------
đang ở trang số: 3
Đã thu thập được 80 liên kết công việc.
------------------------------------
đang ở trang số: 4
Đã thu thập được 100 liên kết công việc.
------------------------------------
đang ở trang số: 4
Đã thu thập được 100 liên kết công việc.


KeyboardInterrupt: 

In [14]:
driver =setup_driver()
sign_in(driver, "dihidromonooxit01012000@gmail.com", "1!Aaaaaaaaaa")

Đã load cookies từ cookies\itviec_cookies.pkl
Đã đăng nhập thành công bằng cookies!


True

Hàm phân tách lấy các dữ liệu cần thiết của 1 job

In [19]:
def parse_detail_job(driver, link):
    driver.get(link)
    time.sleep(random.uniform(3, 5))
    content = driver.page_source
    
    soup = BeautifulSoup(content, 'html.parser')
    job_title = soup.find('div', class_='job-header-info').find('h1').get_text(strip=True)
    company_name = soup.find('div', class_='employer-name').get_text(strip=True)
    salary = soup.find('div', class_='salary').get_text(strip=True)
    
    
    
    job_show_info = soup.find('div', class_='job-show-info').find_all('span')
    job_show_info_list = [info.get_text().replace('\n',' ') for info in job_show_info]
    company_location  = job_show_info_list[0]
    woring_model = job_show_info_list[1]
    post_time = job_show_info_list[2]
    #skill and job_expertise
    skills, job_expertise = soup.find('div', class_='job-show-info').find_all('div', class_='imb-4 imb-xl-3 d-flex flex-column flex-xl-row igap-3 align-items-xl-baseline')
    skills_list = [skill.get_text(strip=True) for skill in skills.find_all('a')]
    job_expertise_list = [exp.get_text(strip=True) for exp in job_expertise.find_all('a')]
    
    #job_domain
    job_domain = soup.find_all('div', class_='itag bg-light-grey itag-sm cursor-default')
    job_domain = [domain.get_text(strip=True) for domain in job_domain]
    
    #job description, your_skill_and_experience, benefit
    job_description, your_skill_and_experience, benefit = soup.find_all('div', class_='imy-5 paragraph')
    job_description = job_description.get_text(strip=True, separator='\n')
    your_skill_and_experience = your_skill_and_experience.get_text(strip=True, separator='\n')
    benefit = benefit.get_text(strip=True, separator='\n')
    
    # company detail với pattern để chỉ chọn đúng class
    company_detail = {}
    pattern_1 = re.compile(r'\b' + re.escape('job-show-employer-info') + r'\b')
    job_show_employer_info = soup.find('section', class_=pattern_1)
    
    if job_show_employer_info:
        company_detail['company_name'] = job_show_employer_info.find('h3').get_text(strip=True)
        company_detail['declaration'] = job_show_employer_info.find('div', class_='imt-5 imt-xl-4').get_text(strip=True)
        
        pattern_2 = re.compile(r'\b' + re.escape('row ipy-2') + r'\b')
        info_items = job_show_employer_info.find_all('div', class_=pattern_2)
        for item in info_items:
            item = item.get_text(strip=True, separator='\n')
            key = item.split('\n')[0]
            value = item.split('\n')[1]
            company_detail[key] = value
    
    return {
        'job_title': job_title,
        'company_name': company_name,
        'salary': salary,
        'company_location': company_location,
        'woring_model': woring_model,
        'post_time': post_time,
        'skills_list': skills_list,
        'job_expertise': job_expertise_list,
        'job_domain': job_domain,
        'job_description': job_description,
        'your_skill_and_experience': your_skill_and_experience,
        'benefit': benefit,
        'company_detail': company_detail,
        
    }
    

In [ ]:
parse_detail_job(driver, 'https://itviec.com/it-jobs/remote-amazing-graphics-designer-softly-llc-softly-llc-1950')

{'job_title': '[HN] Business Analyst',
 'company_name': 'Giao Hàng Tiết Kiệm',
 'salary': 'Sign in to view salary',
 'company_location': 'GHTK Building, đường Phạm Hùng, phường Mễ Trì, quận Nam Từ Liêm, thành phố Hà Nội, Nam Tu Liem, Ha Noi',
 'woring_model': ' Fresher Accepted ',
 'post_time': 'At office',
 'skills_list': ['Business Analysis',
  'Power BI',
  'Postman',
  'User diagram',
  'Figma'],
 'job_expertise': ['Business Analyst'],
 'job_domain': ['E-commerce', 'Software Products and Web Services'],
 'job_description': 'Job description\nChuyên viên phân tích nghiệp vụ sản phẩm sẽ làm việc với các phòng ban khác nhau (Phòng hoạt hành, Trung tâm công nghệ, Chăm sóc khách hàng, nhóm thiết kế,..) để tối ưu hóa sản phẩm mang lại nhiều sản phẩm có giá trị hơn, tiết kiệm chi phí, trả lại lợi nhuận và tăng sự hài lòng của khách hàng và nhân viên nội bộ:\n– Tạo và duy trì trình phát triển sản phẩm bao gồm nghiên cứu sản phẩm, nghiên cứu thị trường, phân tích đối thủ cạnh tranh, lập kế h

In [ ]:
def run_crawl(save_to_file=True, output_dir='.', email='dihidromonooxit01012000@gmail.com', password='1!Aaaaaaaaaa', format='jsonl'):
    """
    Chạy crawler chính, trả về danh sách job và lưu kết quả.
    
    Args:
        save_to_file (bool): Có lưu kết quả ra file hay không
        output_dir (str): Thư mục để lưu file output
        email (str): Email đăng nhập
        password (str): Mật khẩu đăng nhập
        format (str): Định dạng file output ('json' hoặc 'jsonl')
    """
    driver = setup_driver()
    job_details = []
    timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Chuẩn bị file output nếu cần
    if save_to_file:
        file_ext = 'jsonl' if format == 'jsonl' else 'json'
        filename = os.path.join(output_dir, f'itviec_jobs_{timestamp}.{file_ext}')
        
        # Tạo thư mục output nếu chưa tồn tại
        os.makedirs(output_dir, exist_ok=True)
        
        if format == 'jsonl':
            # Tạo file trống để sẵn sàng append
            with open(filename, 'w', encoding='utf-8') as f:
                pass
    
    # Đăng nhập và kiểm tra
    try:
        if not sign_in(driver, email, password):
            print("Không thể tiếp tục vì đăng nhập thất bại.")
            driver.quit()
            return []
    except Exception as e:
        print(f'Lỗi nghiêm trọng trong quá trình đăng nhập: {e}')
        driver.quit()
        return []

    try:
        # Thu thập links
        links = crawl_links(driver)
        print(len(links), 'links đã được thu thập.')
        
        # Thu thập chi tiết từng job
        for idx, link in enumerate(links, 1):
            try:
                detail = parse_detail_job(driver, link)
                job_details.append(detail)
                print(f"[{idx}/{len(links)}] Đã parse thành công: {detail.get('job_title', 'Unknown Title')}")
                
                # Nếu format là JSONL, ghi ngay job vừa crawl được
                if save_to_file and format == 'jsonl':
                    with open(filename, 'a', encoding='utf-8') as f:
                        f.write(json.dumps(detail, ensure_ascii=False) + '\n')
                
            except Exception as e:
                print(f'[{idx}/{len(links)}] Lỗi khi parse {link}: {e}. Bỏ qua.')
                continue

        # Nếu format là JSON, đợi crawl xong hết mới ghi file
        if save_to_file and format != 'jsonl' and job_details:
            try:
                with open(filename, 'w', encoding='utf-8') as f:
                    json.dump(job_details, f, ensure_ascii=False, indent=2)
                print(f'Đã lưu {len(job_details)} job vào {filename}')
            except Exception as e:
                print(f'Không thể lưu file JSON: {e}')

        return job_details

    finally:
        # Đảm bảo driver luôn được đóng
        try:
            driver.quit()
            print("Đã đóng trình duyệt.")
        except:
            pass

# Runner ví dụ: gọi run_crawl và lưu kết quả theo định dạng JSONL
if __name__ == '__main__':
    # Bạn có thể đổi output_dir thành đường dẫn lưu mong muốn
    # format='jsonl' để lưu theo định dạng JSONL (mặc định)
    # hoặc format='json' để lưu theo định dạng JSON Array
    jobs = run_crawl(save_to_file=True, output_dir='.', format='jsonl')
    print('Hoàn tất. Số job thu thập được:', len(jobs))

Đăng nhập thành công!
------------------------------------
đang ở trang số: 0
Đã thu thập được 20 liên kết công việc.
------------------------------------
đang ở trang số: 0
Đã thu thập được 20 liên kết công việc.
------------------------------------
đang ở trang số: 1
Đã thu thập được 40 liên kết công việc.
------------------------------------
đang ở trang số: 1
Đã thu thập được 40 liên kết công việc.
------------------------------------
đang ở trang số: 2
Đã thu thập được 60 liên kết công việc.
------------------------------------
đang ở trang số: 2
Đã thu thập được 60 liên kết công việc.
------------------------------------
đang ở trang số: 3
Đã thu thập được 80 liên kết công việc.
------------------------------------
đang ở trang số: 3
Đã thu thập được 80 liên kết công việc.
------------------------------------
đang ở trang số: 4
Đã thu thập được 100 liên kết công việc.
------------------------------------
đang ở trang số: 4
Đã thu thập được 100 liên kết công việc.
----------------

# Xử lý thêm việc chuyển từ json array sang jsonl
Không cần dùng nữa vì đã lưu thành jsonl trong hàm run_crawl


In [ ]:
# import json

# def convert_json_array_to_jsonl(input_file_path, output_file_path):
#     """
#     Đọc một file JSON chứa mảng (Array) và ghi từng đối tượng ra file JSONL.
#     """
#     try:
#         # 1. Đọc toàn bộ nội dung file JSON Array
#         with open(input_file_path, 'r', encoding='utf-8') as f:
#             # Tải nội dung thành một List/Array trong Python
#             data_array = json.load(f)

#         # 2. Ghi từng đối tượng trong Array ra file JSONL
#         with open(output_file_path, 'w', encoding='utf-8') as f_out:
#             # Lặp qua từng đối tượng (dictionary) trong list
#             for record in data_array:
#                 # Chuyển đối tượng thành chuỗi JSON và thêm ký tự xuống dòng
#                 json_line = json.dumps(record, ensure_ascii=False)
#                 f_out.write(json_line + '\n')

#         print(f"✅ Chuyển đổi thành công! Dữ liệu đã được lưu tại: {output_file_path}")

#     except FileNotFoundError:
#         print(f"❌ Lỗi: Không tìm thấy file tại đường dẫn {input_file_path}")
#     except json.JSONDecodeError:
#         print("❌ Lỗi: File JSON đầu vào không hợp lệ.")
#     except Exception as e:
#         print(f"❌ Đã xảy ra lỗi: {e}")

# # --- Sử dụng hàm ---
# input_file = 'itviec_jobs_20251024_204758.json'  # Tên file JSON Array của bạn
# output_file = 'itviec_jobs.jsonl' # Tên file JSON Lines mong muốn



# convert_json_array_to_jsonl(input_file, output_file)

✅ Chuyển đổi thành công! Dữ liệu đã được lưu tại: itviec_jobs.jsonl


In [24]:
def test(driver, email = 'dihidromonooxit01012000@gmail.com', password = '1!Aaaaaaaaaa'):
    driver.get("https://itviec.com/sign_in")
    time.sleep(3)

    email_input = driver.find_element(By.NAME, "user[email]")
    password_input = driver.find_element(By.NAME, "user[password]")
    submit_button = driver.find_element(By.XPATH, "//button[@type='submit']")

    email_input.send_keys(email)
    password_input.send_keys(password)
    submit_button.click()
    time.sleep(5)

    # Quay về trang chủ để đảm bảo cookie đã lưu vào domain chính
    # driver.get("https://itviec.com")
    # time.sleep(3)
    # save_cookies(driver, 'cookies/itviec_cookies.pkl')
    driver.get('https://itviec.com/it-jobs')
    load_cookies(driver, 'cookies/itviec_cookies.pkl')
    time.sleep(3)
    driver.refresh()
    time.sleep(5)
    # return driver.get_cookies()
    content = driver.page_source
    soup = BeautifulSoup(content, 'html.parser')
    # avatar = soup.find('div', class_='sign-in-user-avatar')
    # print(avatar)
    # return avatar
    return content
    

In [25]:
driver =setup_driver()
test(driver)

Đã load cookies từ cookies/itviec_cookies.pkl


'<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml"><head><style type="text/css" data-tag-name="trix-editor">trix-editor {\n    display: block;\n}\n\ntrix-editor:empty::before {\n    content: attr(placeholder);\n    color: graytext;\n    cursor: text;\n    pointer-events: none;\n    white-space: pre-line;\n}\n\ntrix-editor a[contenteditable=false] {\n    cursor: text;\n}\n\ntrix-editor img {\n    max-width: 100%;\n    height: auto;\n}\n\ntrix-editor [data-trix-attachment] figcaption textarea {\n    resize: none;\n}\n\ntrix-editor [data-trix-attachment] figcaption textarea.trix-autoresize-clone {\n    position: absolute;\n    left: -9999px;\n    max-height: 0px;\n}\n\ntrix-editor [data-trix-attachment] figcaption[data-trix-placeholder]:empty::before {\n    content: attr(data-trix-placeholder);\n    color: graytext;\n}\n\ntrix-editor [data-trix-cursor-target] {\n    display: inline-block !important;\n    width: 1px !important;\n    padding: 0 !important;\n    margin: 0 !i

In [3]:
def test2(driver):
    driver.get("https://itviec.com/it-jobs")
    load_cookies(driver, 'cookies/itviec_cookies.pkl')
    # driver.get('https://itviec.com/it-jobs?job_selected=remote-amazing-graphics-designer-softly-llc-softly-llc-1950')
    time.sleep(2)
    driver.refresh()
    time.sleep(2)
    content = driver.page_source
    soup = BeautifulSoup(content, 'html.parser')
    avatar = soup.find('div', class_='sign-in-user-avatar')
    print(avatar)
    return avatar
    # return translate_button

In [6]:
driver = setup_driver()
test2(driver)

Đã load cookies từ cookies/itviec_cookies.pkl
None
None


In [ ]:
def test4(driver):
    driver.get()